In [10]:
import pandas as pd
import numpy as np
import random
from urllib import parse
from sqlalchemy import create_engine
from datetime import timedelta
from unidecode import unidecode

In [11]:
games = pd.read_csv("data/games_to_choose.csv") 
male_names = pd.read_csv("data/male_names.csv")
female_names = pd.read_csv("data/female_names.csv")
male_surnames = pd.read_csv("data/male_surnames.csv")
female_surnames = pd.read_csv("data/female_surnames.csv")

In [12]:
addresses = pd.read_excel("data/addresses.xlsx", engine = "openpyxl")

In [13]:
engine = create_engine("mysql+pymysql://{user}:{password}@{account}:3306/team03".format(
    user = "team03",
    password = parse.quote("te@m0e"),
    account = "giniewicz.it",
    )
)

In [14]:
def choose_games(games_to_choose, num_of_games):
    """
    Returns list with ID of games which the shop will be able to sale or rent.
    
    Arguments
    ---------
    games_to_choose_from_df - data frame with all games
    num_of_games - num of games' titles that the shop will be able to sale or rent
    """
    tournament_games = games_to_choose[games_to_choose["Tournament_game"] == 1]
    not_tournament_games = games_to_choose[games_to_choose["Tournament_game"] == 0]
    games_sample = not_tournament_games.sample(num_of_games - 5)
    not_tournament_id = list(games_sample["Game_ID"])
    tournament_id = list(tournament_games["Game_ID"])
    all_games = not_tournament_id + tournament_id
    return all_games

In [15]:
all_games = choose_games(games, 50) #game

In [16]:
def choose_products(games_to_choose_from, num_of_products):
    """
    Returns data frame with products that are available on the first day.
    
    Arguments
    ---------
    games_to_choose_from_df - data frame with all games that the shop will be able to sale or rent
    num_of_products - num of productst that the shop will stock on the first day
    """
    
    not_tournament_id = games_to_choose_from[-5:]
    tournament_products = [not_tournament_id[i] for i in np.arange(5)]*5
    for_rent_tournament = np.zeros(25)
    for_sale_tournament = np.zeros(25)
    for_tournament_tournament = [1 for i in np.arange(25)]
    
    not_tournament_products = random.choices(games_to_choose_from, k=num_of_products - 25)
    for_tournament_nt = [0 for i in np.arange(num_of_products - 25)]
    supply_date= ['2022-04-29' for i in np.arange(num_of_products)]
    
    for_sale_nt = np.zeros(num_of_products - 25)
    for_rent_nt = np.zeros(num_of_products - 25)

    for i in np.arange(num_of_products - 25):
        rand = random.randint(0, 1)
        for_sale_nt[i] = rand
        if for_sale_nt[i] == 0:
            for_rent_nt[i] = 1
        else:
            for_rent_nt[i] = 0
            
    for_rent = list(for_rent_tournament) + list(for_rent_nt)
    for_sale = list(for_sale_tournament) + list(for_sale_nt)
    for_tournament = for_tournament_tournament + for_tournament_nt
    all_games_products = tournament_products + not_tournament_products
    
    product_df = pd.DataFrame()
    product_df["Product_ID"] = np.arange(num_of_products)
    product_df["For_rent"] = for_rent
    product_df["For_sale"] = for_sale
    product_df["For_tournament"] = for_tournament
    product_df["Supply_date"] = supply_date
    product_df["Game_ID"] = all_games_products
    product_df["Purchase_ID"] = np.nan

    product_df["Supply_date"] = pd.to_datetime(product_df["Supply_date"])
    product_df["For_rent"] = pd.to_numeric(product_df["For_rent"], downcast='integer')
    product_df["For_sale"] = pd.to_numeric(product_df["For_sale"], downcast='integer')
    
    return product_df

In [17]:
products_df = choose_products(all_games, 500)

In [18]:
def product_df_time(product_df, first_date): 
    
    cust_num = 101 #real customer number is cust_num - 1
    worker_num = 4 #real employee number is worker_num - 1
    
    rental_id = [0]
    rental_date = []
    return_date_expected = []
    return_date_actual = []
    customer_rent_id = []
    worker_rent_id = []
    
    relation_id = [0]
    product_id_rel = []
    rental_id_rel = []
    
    product_and_return = {}
    
    purchase_id = [0]
    purchase_date = []
    customer_purch_id = []
    worker_purch_id = []  
    
    tournament_id = [0]
    tournament_date = []
    ticket_price = []
    game_id = []
    tournament_cost = []
    
    tournament_id_score = []
    customer_id_score = []
    score = []
    
    num_to_rent = np.random.poisson(10, 365)
    num_to_sale = np.random.poisson(10, 365)
    
    num_to_supply = np.random.poisson(10, 365)
    
    j = 0
    for days_count in np.arange(1, 366):
        
        if days_count % 7 != 2:

            new_date = first_date + timedelta(days = int(days_count))

            games_rentable = product_df[product_df["For_rent"] == 1]

            num_of_games_to_rent = np.random.poisson(1/1.3, num_to_rent[j]) #ile gier na wypożyczenie?

            while sum(num_of_games_to_rent) > len(games_rentable): 
                num_of_games_to_rent = num_of_games_to_rent[:-1]

            random_return = np.random.random(len(num_of_games_to_rent))
            rent_cust_id = np.random.randint(1, cust_num, len(num_of_games_to_rent))
            rent_worker_id = np.random.randint(1, worker_num, len(num_of_games_to_rent))

            games_to_rent = games_rentable["Product_ID"].sample(sum(num_of_games_to_rent)) #jakie gry wypożyczono?

            k = 0
            i = 0
            for rent in num_of_games_to_rent:

                games_to_this_rent = np.array(games_to_rent)[k:k+rent]

                available_products = []

                for game in games_to_this_rent: #sprawdzamy, czy produkty są dostępne
                    if game not in list(product_and_return.keys()) or product_and_return[game] < new_date:
                        available_products.append(game)

                num_of_products = len(available_products)


                if num_of_products > 0:

                    rental_id.append(rental_id[-1]+1)
                    rental_date.append(new_date)
                    return_date_expected.append(new_date + timedelta(days = 3))
                    customer_rent_id.append(rent_cust_id[i])
                    worker_rent_id.append(rent_worker_id[i])

                    if random_return[i] < 0.5:
                        return_date_actual.append(return_date_expected[-1])
                        product_and_return[game] = return_date_expected[-1]

                    else:
                        return_date = new_date + timedelta(days = np.random.geometric(1/3))
                        return_date_actual.append(return_date)
                        product_and_return[game] = return_date

                    relation_id = relation_id + [relation_id[-1] + i for i in np.arange(1, num_of_products+1)]
                    rental_id_rel = rental_id_rel + [rental_id[-1]]*num_of_products
                    product_id_rel = product_id_rel + available_products

                k += rent
                i += 1


            #zakup gier

            games_salable = product_df[(product_df["For_sale"] == 1) & (pd.isna(product_df["Purchase_ID"]))]
            num_of_games_to_sale = np.random.geometric(1/1.3, num_to_sale[j]) #ile gier na wypożyczenie?
            purch_cust_id = np.random.randint(1, cust_num, len(num_of_games_to_sale))
            purch_worker_id = np.random.randint(1, worker_num, len(num_of_games_to_sale))
            random_cust = np.random.random(len(num_of_games_to_sale))

            while sum(num_of_games_to_sale) > len(games_salable): 
                num_of_games_to_sale = num_of_games_to_sale[:-1]

            games_to_sale = games_salable["Product_ID"].sample(sum(num_of_games_to_sale)) #jakie gry wypożyczono?

            k = 0
            i = 0

            for purchase in num_of_games_to_sale:

                games_to_this_purchase = np.array(games_to_sale)[k:k+purchase] 

                purchase_id.append(purchase_id[-1]+1)
                purchase_date.append(new_date)

                if random_cust[i] < 0.3:
                    customer_purch_id.append(purch_cust_id[i])
                else:
                    customer_purch_id.append(np.nan)

                worker_purch_id.append(purch_worker_id[i])

              
                for game_product in games_to_this_purchase:
                    product_df.loc[game_product, "Purchase_ID"] = purchase_id[-1]

                k += purchase
                i += 1

            games_to_supply = random.choices(all_games, k = num_to_supply[j])

            max_product_id = max(np.array(product_df["Product_ID"]))
            new_df = pd.DataFrame()
            new_df["Product_ID"] = np.arange(max_product_id + 1, max_product_id + num_to_supply[j] + 1)
            new_df["For_sale"] = [1 for _ in np.arange(num_to_supply[j])]
            new_df["For_rent"] = [0 for _ in np.arange(num_to_supply[j])]
            new_df["For_tournament"] = [0 for _ in np.arange(num_to_supply[j])]
            new_df["Supply_date"] = [new_date for _ in np.arange(num_to_supply[j])]
            new_df["Game_ID"] = games_to_supply
            new_df["Purchase_ID"] = np.nan

            new_df["Supply_date"] = pd.to_datetime(new_df["Supply_date"])
            new_df["For_rent"] = pd.to_numeric(new_df["For_rent"], downcast='integer')
            new_df["For_sale"] = pd.to_numeric(new_df["For_sale"], downcast='integer')


            product_df = pd.concat([product_df, new_df], ignore_index=True)  
            
            #turnieje gier
            
            if days_count % 7 == 1 or days_count % 7 == 0:
                if np.random.random() < 0.3:
                    tournament_id.append(tournament_id[-1] + 1)
                    tournament_date.append(new_date)
                    ticket_price.append(np.random.poisson(18))
                    game_id.append(np.random.randint(1, 6))
                    tournament_cost.append(np.random.poisson(125))
                    
                    num_of_players = np.max([10, np.random.poisson(25)])
                    
                    while num_of_players >= cust_num - 1:
                        num_of_players =- 1
                        
                    if num_of_players > 0:
                        
                        customer_id_score = customer_id_score + list(np.random.choice(np.arange(cust_num), num_of_players, replace=False))
                        score = score + list(np.arange(num_of_players))
                        tournament_id_score = tournament_id_score + [tournament_id[-1] for _ in np.arange(num_of_players)]
                    

        j += 1
            
            

    score_id = np.arange(len(score))
    
    rental_df = pd.DataFrame({"Rental_ID":rental_id[1:], "Rental_date":rental_date, "Return_date_expected":return_date_expected, "Return_date_actual":return_date_actual, "Customer_ID":customer_rent_id, "Employee_ID":worker_rent_id})
    rental_product_rel = pd.DataFrame({"Relation_ID":relation_id[1:], "Product_ID":product_id_rel, "Rental_ID":rental_id_rel})
    purchase_df = pd.DataFrame({"Purchase_ID":purchase_id[1:], "Purchase_date":purchase_date, "Customer_ID":customer_purch_id, "Employee_ID":worker_purch_id})
    tournament_df = pd.DataFrame({"Tournament_ID":tournament_id[1:], "Tournament_date":tournament_date, "Ticket_price":ticket_price, "Tournament_cost":tournament_cost, "Game_ID":game_id})
    score_df = pd.DataFrame({"Score_ID":score_id, "Tournament_ID":tournament_id_score, "Customer_ID":customer_id_score, "Score":score})
    
    return rental_df, rental_product_rel, purchase_df, product_df, tournament_df, score_df

In [19]:
rental_df, rental_product_rel, purchase_df, product_df, tournament_df, score_df = product_df_time(products_df, products_df["Supply_date"].loc[0])

In [23]:
product_df

,Product_ID,For_rent,For_sale,For_tournament,Supply_date,Game_ID,Purchase_ID
0,0,0,0,1,2022-04-29,68448,NaN
1,1,0,0,1,2022-04-29,822,NaN
2,2,0,0,1,2022-04-29,39856,NaN
3,3,0,0,1,2022-04-29,13,NaN
4,4,0,0,1,2022-04-29,148228,NaN
...,...,...,...,...,...,...,...
3609,3609,0,1,0,2023-04-29,822,NaN
3610,3610,0,1,0,2023-04-29,322865,NaN
3611,3611,0,1,0,2023-04-29,266936,NaN
3612,3612,0,1,0,2023-04-29,822,NaN


In [20]:
tournament_df

,Tournament_ID,Tournament_date,Ticket_price,Tournament_cost,Game_ID
0,1,2022-05-06,13,128,1
1,2,2022-06-11,15,123,5
2,3,2022-07-02,23,116,4
3,4,2022-07-08,14,103,1
4,5,2022-07-09,15,115,5
5,6,2022-07-15,15,125,5
6,7,2022-08-13,11,121,2
7,8,2022-08-27,19,125,3
8,9,2022-09-03,14,136,3
9,10,2022-09-16,24,131,3


In [21]:
rental_product_rel.set_index("Relation_ID", inplace = True)

In [22]:
rental_product_rel.to_sql("rental_product_rel", engine, if_exists = "append")

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '2' for key 'PRIMARY'")
[SQL: INSERT INTO rental_product_rel (`Relation_ID`, `Product_ID`, `Rental_ID`) VALUES (%(Relation_ID)s, %(Product_ID)s, %(Rental_ID)s)]
[parameters: ({'Relation_ID': 1, 'Product_ID': 74, 'Rental_ID': 1}, {'Relation_ID': 2, 'Product_ID': 295, 'Rental_ID': 2}, {'Relation_ID': 3, 'Product_ID': 203, 'Rental_ID': 2}, {'Relation_ID': 4, 'Product_ID': 336, 'Rental_ID': 3}, {'Relation_ID': 5, 'Product_ID': 393, 'Rental_ID': 4}, {'Relation_ID': 6, 'Product_ID': 320, 'Rental_ID': 4}, {'Relation_ID': 7, 'Product_ID': 37, 'Rental_ID': 4}, {'Relation_ID': 8, 'Product_ID': 455, 'Rental_ID': 5}  ... displaying 10 of 2368 total bound parameter sets ...  {'Relation_ID': 2367, 'Product_ID': 299, 'Rental_ID': 1664}, {'Relation_ID': 2368, 'Product_ID': 125, 'Rental_ID': 1665})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [ ]:
rental_df.set_index("Rental_ID", inplace = True)

In [ ]:
rental_df.to_sql("Rental", engine, if_exists = "replace")

1608

In [ ]:
purchase_df    

,Purchase_ID,Purchase_date,Customer_ID,Employee_ID
0,1,2022-04-30,55.0,3
1,2,2022-04-30,10.0,3
2,3,2022-04-30,NaN,2
3,4,2022-04-30,11.0,2
4,5,2022-04-30,NaN,1
...,...,...,...,...
2645,2646,2023-04-29,NaN,1
2646,2647,2023-04-29,66.0,3
2647,2648,2023-04-29,4.0,2
2648,2649,2023-04-29,13.0,2


In [ ]:
product_df[~product_df.Purchase_ID.isna()].groupby("Game_ID").count()

,Product_ID,For_rent,For_sale,For_tournament,Supply_date,Purchase_ID
Game_ID,,,,,,
13,64,64,64,64,64,64
822,69,69,69,69,69,69
1927,74,74,74,74,74,74
5716,79,79,79,79,79,79
30549,70,70,70,70,70,70
36634,60,60,60,60,60,60
39856,52,52,52,52,52,52
68448,78,78,78,78,78,78
85325,59,59,59,59,59,59


In [ ]:
tournament_df

,Tournament_ID,Tournament_date,Ticket_price,Tournament_cost,Game_ID
0,1,2022-05-28,28,133,1
1,2,2022-06-10,25,146,1
2,3,2022-06-25,20,130,3
3,4,2022-07-15,23,118,4
4,5,2022-07-16,12,134,1
5,6,2022-07-23,18,131,4
6,7,2022-07-30,12,130,4
7,8,2022-08-05,18,122,3
8,9,2022-08-12,17,117,4
9,10,2022-09-10,17,133,4
